# 我們使用多層感知器完成 MNIST 手寫體數字圖片資料集 [LeCun1998] 的分類任務。
<img src="https://tf.wiki/_images/mnist_0-9.png">

# 資料獲取及預處理： tf.keras.datasets
## 先進行預備工作，實現一個簡單的 MNISTLoader 類來讀取 MNIST 資料集資料。這裡使用了 tf.keras.datasets 快速載入 MNIST 資料集。

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
class MNISTLoader():
    def __init__(self):
        mnist = tf.keras.datasets.mnist
        (self.train_data, self.train_label), (self.test_data, self.test_label) = mnist.load_data()
        # MNIST中的圖片預設為uint8（0-255的數字）。以下程式碼將其正規化到0-1之間的浮點數，並在最後增加一維作為顏色通道
        self.train_data = np.expand_dims(self.train_data.astype(np.float32) / 255.0, axis=-1)      # [60000, 28, 28, 1]
        self.test_data = np.expand_dims(self.test_data.astype(np.float32) / 255.0, axis=-1)        # [10000, 28, 28, 1]
        self.train_label = self.train_label.astype(np.int32)    # [60000]
        self.test_label = self.test_label.astype(np.int32)      # [10000]
        self.num_train_data, self.num_test_data = self.train_data.shape[0], self.test_data.shape[0]

    def get_batch(self, batch_size):
        # 從資料集中隨機取出batch_size個元素並返回
        index = np.random.randint(0, self.num_train_data, batch_size)
        return self.train_data[index, :], self.train_label[index]

In [3]:
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()    # Flatten層將除第一維（batch_size）以外的維度展平
        self.dense1 = tf.keras.layers.Dense(units=100, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=10)

    def call(self, inputs):         # [batch_size, 28, 28, 1]
        x = self.flatten(inputs)    # [batch_size, 784]
        x = self.dense1(x)          # [batch_size, 100]
        x = self.dense2(x)          # [batch_size, 10]
        output = tf.nn.softmax(x)   # 正規化output
        return output

<img src="https://tf.wiki/_images/mlp.png" height="50%">
MLP模型示意圖

## 模型的訓練： tf.keras.losses 和 tf.keras.optimizer
### 定義一些模型超參數 (Hyperparameters)：

In [11]:
num_epochs = 5
batch_size = 50
learning_rate = 0.001

In [12]:
model = MLP()
data_loader = MNISTLoader()
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

### 然後疊代進行以下步驟：

1.從 DataLoader 中隨機取一批訓練資料；

2.將這批資料送入模型，計算出模型的預測值；

3.將模型預測值與真實值進行比較，計算損失函數（loss）。這里使用 tf.keras.losses 中的交叉熵函數作為損失函數；

4.計算損失函數關於模型變數的導數；

5.將求出的導數值傳入優化器，使用優化器的 apply_gradients 方法更新模型參數以最小化損失函數

In [13]:
num_batches = int(data_loader.num_train_data // batch_size*num_epochs)
for batch_index in range(num_batches):
    X, y = data_loader.get_batch(batch_size)
    with tf.GradientTape() as tape:
        y_pred = model(X)
        loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y, y_pred=y_pred)
        loss = tf.reduce_mean(loss)
        print("batch %d: loss %f" % (batch_index, loss.numpy()))
    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))

batch 0: loss 2.343445
batch 1: loss 2.305157
batch 2: loss 2.215088
batch 3: loss 2.119383
batch 4: loss 2.097402
batch 5: loss 1.908825
batch 6: loss 1.905434
batch 7: loss 1.827699
batch 8: loss 1.800125
batch 9: loss 1.680600
batch 10: loss 1.655579
batch 11: loss 1.636289
batch 12: loss 1.590713
batch 13: loss 1.355454
batch 14: loss 1.437530
batch 15: loss 1.337723
batch 16: loss 1.129154
batch 17: loss 1.430872
batch 18: loss 1.275735
batch 19: loss 1.339703
batch 20: loss 1.136244
batch 21: loss 1.119534
batch 22: loss 1.206621
batch 23: loss 0.954643
batch 24: loss 1.084488
batch 25: loss 1.076814
batch 26: loss 0.861098
batch 27: loss 0.905512
batch 28: loss 0.974973
batch 29: loss 0.782491
batch 30: loss 0.733791
batch 31: loss 0.854856
batch 32: loss 0.694693
batch 33: loss 0.849602
batch 34: loss 0.801497
batch 35: loss 0.776183
batch 36: loss 0.742186
batch 37: loss 0.633664
batch 38: loss 0.808013
batch 39: loss 0.532384
batch 40: loss 0.666045
batch 41: loss 0.727817
ba

batch 364: loss 0.331084
batch 365: loss 0.282932
batch 366: loss 0.248067
batch 367: loss 0.184365
batch 368: loss 0.194255
batch 369: loss 0.303322
batch 370: loss 0.234361
batch 371: loss 0.347015
batch 372: loss 0.347807
batch 373: loss 0.156063
batch 374: loss 0.241154
batch 375: loss 0.266500
batch 376: loss 0.125412
batch 377: loss 0.319896
batch 378: loss 0.366265
batch 379: loss 0.440318
batch 380: loss 0.158226
batch 381: loss 0.449575
batch 382: loss 0.087400
batch 383: loss 0.296743
batch 384: loss 0.196335
batch 385: loss 0.183106
batch 386: loss 0.189441
batch 387: loss 0.139054
batch 388: loss 0.345489
batch 389: loss 0.323314
batch 390: loss 0.452879
batch 391: loss 0.470407
batch 392: loss 0.180079
batch 393: loss 0.161632
batch 394: loss 0.240532
batch 395: loss 0.432930
batch 396: loss 0.337710
batch 397: loss 0.331920
batch 398: loss 0.150552
batch 399: loss 0.334850
batch 400: loss 0.267224
batch 401: loss 0.167524
batch 402: loss 0.282626
batch 403: loss 0.180601


batch 708: loss 0.297624
batch 709: loss 0.151970
batch 710: loss 0.186009
batch 711: loss 0.245340
batch 712: loss 0.133736
batch 713: loss 0.156454
batch 714: loss 0.276353
batch 715: loss 0.122279
batch 716: loss 0.176379
batch 717: loss 0.241763
batch 718: loss 0.351392
batch 719: loss 0.166777
batch 720: loss 0.244033
batch 721: loss 0.253910
batch 722: loss 0.265861
batch 723: loss 0.114146
batch 724: loss 0.468415
batch 725: loss 0.243541
batch 726: loss 0.198572
batch 727: loss 0.166563
batch 728: loss 0.179748
batch 729: loss 0.408286
batch 730: loss 0.291420
batch 731: loss 0.191177
batch 732: loss 0.191248
batch 733: loss 0.146828
batch 734: loss 0.094057
batch 735: loss 0.155070
batch 736: loss 0.149466
batch 737: loss 0.084549
batch 738: loss 0.179633
batch 739: loss 0.357020
batch 740: loss 0.094589
batch 741: loss 0.232918
batch 742: loss 0.162974
batch 743: loss 0.294364
batch 744: loss 0.080771
batch 745: loss 0.216042
batch 746: loss 0.216225
batch 747: loss 0.244825


batch 1056: loss 0.178763
batch 1057: loss 0.172378
batch 1058: loss 0.161409
batch 1059: loss 0.191552
batch 1060: loss 0.148292
batch 1061: loss 0.444935
batch 1062: loss 0.107817
batch 1063: loss 0.139757
batch 1064: loss 0.131499
batch 1065: loss 0.279996
batch 1066: loss 0.313587
batch 1067: loss 0.157796
batch 1068: loss 0.099575
batch 1069: loss 0.047903
batch 1070: loss 0.161285
batch 1071: loss 0.368261
batch 1072: loss 0.154525
batch 1073: loss 0.125955
batch 1074: loss 0.250620
batch 1075: loss 0.173193
batch 1076: loss 0.146282
batch 1077: loss 0.392396
batch 1078: loss 0.163986
batch 1079: loss 0.069545
batch 1080: loss 0.345680
batch 1081: loss 0.163719
batch 1082: loss 0.172592
batch 1083: loss 0.187696
batch 1084: loss 0.088754
batch 1085: loss 0.126430
batch 1086: loss 0.111276
batch 1087: loss 0.196365
batch 1088: loss 0.175459
batch 1089: loss 0.095009
batch 1090: loss 0.153207
batch 1091: loss 0.143221
batch 1092: loss 0.229040
batch 1093: loss 0.146275
batch 1094: 

batch 1392: loss 0.106255
batch 1393: loss 0.090853
batch 1394: loss 0.079452
batch 1395: loss 0.075731
batch 1396: loss 0.106974
batch 1397: loss 0.082253
batch 1398: loss 0.254987
batch 1399: loss 0.096936
batch 1400: loss 0.164221
batch 1401: loss 0.184282
batch 1402: loss 0.222495
batch 1403: loss 0.403573
batch 1404: loss 0.135067
batch 1405: loss 0.325099
batch 1406: loss 0.106402
batch 1407: loss 0.147297
batch 1408: loss 0.133715
batch 1409: loss 0.322789
batch 1410: loss 0.132303
batch 1411: loss 0.143626
batch 1412: loss 0.165828
batch 1413: loss 0.099570
batch 1414: loss 0.061180
batch 1415: loss 0.061491
batch 1416: loss 0.230125
batch 1417: loss 0.123318
batch 1418: loss 0.058183
batch 1419: loss 0.049305
batch 1420: loss 0.126870
batch 1421: loss 0.097829
batch 1422: loss 0.245411
batch 1423: loss 0.275938
batch 1424: loss 0.117361
batch 1425: loss 0.248374
batch 1426: loss 0.302637
batch 1427: loss 0.160095
batch 1428: loss 0.191993
batch 1429: loss 0.078384
batch 1430: 

...

batch 4395: loss 0.015673
batch 4396: loss 0.116566
batch 4397: loss 0.050866
batch 4398: loss 0.012593
batch 4399: loss 0.009080
batch 4400: loss 0.036519
batch 4401: loss 0.044849
batch 4402: loss 0.065173
batch 4403: loss 0.148940
batch 4404: loss 0.151544
batch 4405: loss 0.049866
batch 4406: loss 0.054179
batch 4407: loss 0.085330
batch 4408: loss 0.017501
batch 4409: loss 0.004988
batch 4410: loss 0.009344
batch 4411: loss 0.074633
batch 4412: loss 0.111730
batch 4413: loss 0.070420
batch 4414: loss 0.080555
batch 4415: loss 0.023033
batch 4416: loss 0.056368
batch 4417: loss 0.059386
batch 4418: loss 0.021768
batch 4419: loss 0.024976
batch 4420: loss 0.018121
batch 4421: loss 0.154237
batch 4422: loss 0.008473
batch 4423: loss 0.088937
batch 4424: loss 0.217903
batch 4425: loss 0.020082
batch 4426: loss 0.080023
batch 4427: loss 0.271925
batch 4428: loss 0.016177
batch 4429: loss 0.058754
batch 4430: loss 0.253067
batch 4431: loss 0.126419
batch 4432: loss 0.037116
batch 4433: 

batch 4747: loss 0.110592
batch 4748: loss 0.011568
batch 4749: loss 0.123712
batch 4750: loss 0.112282
batch 4751: loss 0.051914
batch 4752: loss 0.063199
batch 4753: loss 0.044132
batch 4754: loss 0.059509
batch 4755: loss 0.034755
batch 4756: loss 0.150543
batch 4757: loss 0.234621
batch 4758: loss 0.036917
batch 4759: loss 0.028277
batch 4760: loss 0.058712
batch 4761: loss 0.171355
batch 4762: loss 0.047605
batch 4763: loss 0.080524
batch 4764: loss 0.040550
batch 4765: loss 0.043029
batch 4766: loss 0.060814
batch 4767: loss 0.045992
batch 4768: loss 0.039748
batch 4769: loss 0.072720
batch 4770: loss 0.024756
batch 4771: loss 0.084894
batch 4772: loss 0.101494
batch 4773: loss 0.019352
batch 4774: loss 0.056770
batch 4775: loss 0.026131
batch 4776: loss 0.070159
batch 4777: loss 0.103536
batch 4778: loss 0.023499
batch 4779: loss 0.177139
batch 4780: loss 0.091606
batch 4781: loss 0.052724
batch 4782: loss 0.011843
batch 4783: loss 0.049259
batch 4784: loss 0.044096
batch 4785: 

batch 5080: loss 0.065297
batch 5081: loss 0.041100
batch 5082: loss 0.026360
batch 5083: loss 0.035150
batch 5084: loss 0.027471
batch 5085: loss 0.013909
batch 5086: loss 0.093660
batch 5087: loss 0.038464
batch 5088: loss 0.054132
batch 5089: loss 0.063884
batch 5090: loss 0.013191
batch 5091: loss 0.163664
batch 5092: loss 0.139370
batch 5093: loss 0.006380
batch 5094: loss 0.067296
batch 5095: loss 0.057718
batch 5096: loss 0.046588
batch 5097: loss 0.010752
batch 5098: loss 0.096241
batch 5099: loss 0.100559
batch 5100: loss 0.021350
batch 5101: loss 0.017795
batch 5102: loss 0.023697
batch 5103: loss 0.013464
batch 5104: loss 0.018897
batch 5105: loss 0.032306
batch 5106: loss 0.016481
batch 5107: loss 0.025954
batch 5108: loss 0.037591
batch 5109: loss 0.129765
batch 5110: loss 0.034546
batch 5111: loss 0.070028
batch 5112: loss 0.046178
batch 5113: loss 0.118857
batch 5114: loss 0.090996
batch 5115: loss 0.041925
batch 5116: loss 0.010492
batch 5117: loss 0.129580
batch 5118: 

batch 5422: loss 0.045519
batch 5423: loss 0.029890
batch 5424: loss 0.021220
batch 5425: loss 0.139205
batch 5426: loss 0.021597
batch 5427: loss 0.069373
batch 5428: loss 0.035982
batch 5429: loss 0.022056
batch 5430: loss 0.026014
batch 5431: loss 0.042669
batch 5432: loss 0.020224
batch 5433: loss 0.008381
batch 5434: loss 0.112824
batch 5435: loss 0.118281
batch 5436: loss 0.012799
batch 5437: loss 0.034567
batch 5438: loss 0.031290
batch 5439: loss 0.028593
batch 5440: loss 0.053875
batch 5441: loss 0.176735
batch 5442: loss 0.086796
batch 5443: loss 0.012535
batch 5444: loss 0.045906
batch 5445: loss 0.163406
batch 5446: loss 0.034364
batch 5447: loss 0.004777
batch 5448: loss 0.166247
batch 5449: loss 0.040091
batch 5450: loss 0.023526
batch 5451: loss 0.035552
batch 5452: loss 0.022952
batch 5453: loss 0.016291
batch 5454: loss 0.039082
batch 5455: loss 0.075926
batch 5456: loss 0.076933
batch 5457: loss 0.046735
batch 5458: loss 0.067871
batch 5459: loss 0.092787
batch 5460: 

batch 5758: loss 0.005855
batch 5759: loss 0.094938
batch 5760: loss 0.049324
batch 5761: loss 0.021686
batch 5762: loss 0.030290
batch 5763: loss 0.012413
batch 5764: loss 0.013121
batch 5765: loss 0.108498
batch 5766: loss 0.125741
batch 5767: loss 0.046813
batch 5768: loss 0.022728
batch 5769: loss 0.026299
batch 5770: loss 0.082056
batch 5771: loss 0.017738
batch 5772: loss 0.022042
batch 5773: loss 0.060629
batch 5774: loss 0.012971
batch 5775: loss 0.011525
batch 5776: loss 0.015120
batch 5777: loss 0.066687
batch 5778: loss 0.099334
batch 5779: loss 0.028504
batch 5780: loss 0.031016
batch 5781: loss 0.057807
batch 5782: loss 0.124334
batch 5783: loss 0.007103
batch 5784: loss 0.035477
batch 5785: loss 0.004027
batch 5786: loss 0.062404
batch 5787: loss 0.024403
batch 5788: loss 0.046129
batch 5789: loss 0.049205
batch 5790: loss 0.044036
batch 5791: loss 0.218491
batch 5792: loss 0.090081
batch 5793: loss 0.019187
batch 5794: loss 0.022421
batch 5795: loss 0.130866
batch 5796: 

In [14]:
sparse_categorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
num_batches = int(data_loader.num_test_data // batch_size)
for batch_index in range(num_batches):
    start_index, end_index = batch_index * batch_size, (batch_index + 1) * batch_size
    y_pred = model.predict(data_loader.test_data[start_index: end_index])
    sparse_categorical_accuracy.update_state(y_true=data_loader.test_label[start_index: end_index], y_pred=y_pred)
print("test accuracy: %f" % sparse_categorical_accuracy.result())

test accuracy: 0.972100


# 卷積神經網路（CNN）
## 卷積神經網路 （Convolutional Neural Network, CNN）是一種結構類似於人類或動物的 視覺系統 的神經網路，包含一個或多個卷積層（Convolutional Layer）、池化層（Pooling Layer）和全連接層（Fully-connected Layer）。

In [2]:
import tensorflow as tf
import numpy as np

In [3]:
class CNN(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = tf.keras.layers.Conv2D(
            filters=32,
            kernel_size=[5,5],
            padding='same',
            activation=tf.nn.relu
        )
        self.pool1 = tf.keras.layers.MaxPool2D(pool_size=[2,2], strides=2)
        self.conv2 = tf.keras.layers.Conv2D(
            filters=64,
            kernel_size=[5,5],
            padding='same',
            activation=tf.nn.relu
        )
        self.pool2 = tf.keras.layers.MaxPool2D(pool_size=[2,2], strides=2)
        self.flatten = tf.keras.layers.Reshape(target_shape=(7*7*64,))
        self.dense1 = tf.keras.layers.Dense(units=1021, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=10)
        
    def call(self,inputs):
        x = self.conv1(inputs)                  # [batch_size, 28, 28, 32] 因為padding,所以一樣是 28*28
        x = self.pool1(x)                       # [batch_size, 14, 14, 32]
        x = self.conv2(x)                       # [batch_size, 14, 14, 64]
        x = self.pool2(x)                       # [batch_size, 7, 7, 64]
        x = self.flatten(x)                     # [batch_size, 7 * 7 * 64]
        x = self.dense1(x)                      # [batch_size, 1024]
        x = self.dense2(x)                      # [batch_size, 10]
        output = tf.nn.softmax(x)
        return output

In [4]:
num_epochs = 5
batch_size = 50
learning_rate = 0.001

In [5]:
model = CNN()
data_loader = MNISTLoader()
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

In [6]:
num_batches = int(data_loader.num_train_data // batch_size*num_epochs)
for batch_index in range(num_batches):
    X, y = data_loader.get_batch(batch_size)
    with tf.GradientTape() as tape:
        y_pred = model(X)
        loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y, y_pred=y_pred)
        loss = tf.reduce_mean(loss)
        print("batch %d: loss %f" % (batch_index, loss.numpy()))
    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))

batch 0: loss 2.301992
batch 1: loss 2.252861
batch 2: loss 2.056064
batch 3: loss 1.918951
batch 4: loss 1.863809
batch 5: loss 1.697950
batch 6: loss 1.387877
batch 7: loss 1.202325
batch 8: loss 1.032042
batch 9: loss 0.962238
batch 10: loss 1.188702
batch 11: loss 0.683741
batch 12: loss 0.885039
batch 13: loss 0.621914
batch 14: loss 0.615025
batch 15: loss 0.433215
batch 16: loss 0.570404
batch 17: loss 0.594898
batch 18: loss 1.123663
batch 19: loss 0.569911
batch 20: loss 0.440406
batch 21: loss 0.390352
batch 22: loss 0.247646
batch 23: loss 0.464620
batch 24: loss 0.381554
batch 25: loss 0.338593
batch 26: loss 0.280472
batch 27: loss 0.497807
batch 28: loss 0.248214
batch 29: loss 0.350446
batch 30: loss 0.319002
batch 31: loss 0.187852
batch 32: loss 0.199341
batch 33: loss 0.372745
batch 34: loss 0.300687
batch 35: loss 0.356787
batch 36: loss 0.204308
batch 37: loss 0.409021
batch 38: loss 0.448690
batch 39: loss 0.183237
batch 40: loss 0.362693
batch 41: loss 0.160682
ba

batch 333: loss 0.183490
batch 334: loss 0.058880
batch 335: loss 0.135339
batch 336: loss 0.028417
batch 337: loss 0.036667
batch 338: loss 0.057280
batch 339: loss 0.017867
batch 340: loss 0.015683
batch 341: loss 0.027919
batch 342: loss 0.018989
batch 343: loss 0.096527
batch 344: loss 0.050875
batch 345: loss 0.349383
batch 346: loss 0.041802
batch 347: loss 0.199525
batch 348: loss 0.036014
batch 349: loss 0.118271
batch 350: loss 0.117506
batch 351: loss 0.098321
batch 352: loss 0.182817
batch 353: loss 0.076551
batch 354: loss 0.113513
batch 355: loss 0.132374
batch 356: loss 0.032745
batch 357: loss 0.061115
batch 358: loss 0.018086
batch 359: loss 0.084283
batch 360: loss 0.024909
batch 361: loss 0.010800
batch 362: loss 0.115546
batch 363: loss 0.288682
batch 364: loss 0.030215
batch 365: loss 0.170993
batch 366: loss 0.052019
batch 367: loss 0.103103
batch 368: loss 0.042634
batch 369: loss 0.058121
batch 370: loss 0.050878
batch 371: loss 0.020332
batch 372: loss 0.133207


batch 661: loss 0.095486
batch 662: loss 0.056365
batch 663: loss 0.139313
batch 664: loss 0.066864
batch 665: loss 0.018397
batch 666: loss 0.091373
batch 667: loss 0.166256
batch 668: loss 0.037790
batch 669: loss 0.012413
batch 670: loss 0.103052
batch 671: loss 0.124603
batch 672: loss 0.048331
batch 673: loss 0.061258
batch 674: loss 0.038489
batch 675: loss 0.049238
batch 676: loss 0.073879
batch 677: loss 0.022750
batch 678: loss 0.062376
batch 679: loss 0.072802
batch 680: loss 0.056737
batch 681: loss 0.148956
batch 682: loss 0.029808
batch 683: loss 0.010326
batch 684: loss 0.093282
batch 685: loss 0.007639
batch 686: loss 0.014126
batch 687: loss 0.143082
batch 688: loss 0.072450
batch 689: loss 0.044500
batch 690: loss 0.008384
batch 691: loss 0.002646
batch 692: loss 0.021889
batch 693: loss 0.111422
batch 694: loss 0.008032
batch 695: loss 0.010461
batch 696: loss 0.031272
batch 697: loss 0.113328
batch 698: loss 0.044892
batch 699: loss 0.025174
batch 700: loss 0.002544


batch 993: loss 0.053284
batch 994: loss 0.011350
batch 995: loss 0.069637
batch 996: loss 0.059255
batch 997: loss 0.041080
batch 998: loss 0.005442
batch 999: loss 0.045544
batch 1000: loss 0.000544
batch 1001: loss 0.046084
batch 1002: loss 0.045594
batch 1003: loss 0.004709
batch 1004: loss 0.001275
batch 1005: loss 0.003973
batch 1006: loss 0.090114
batch 1007: loss 0.036938
batch 1008: loss 0.052447
batch 1009: loss 0.033732
batch 1010: loss 0.088956
batch 1011: loss 0.036419
batch 1012: loss 0.015709
batch 1013: loss 0.189251
batch 1014: loss 0.009185
batch 1015: loss 0.017163
batch 1016: loss 0.033367
batch 1017: loss 0.012206
batch 1018: loss 0.067436
batch 1019: loss 0.006044
batch 1020: loss 0.116681
batch 1021: loss 0.017242
batch 1022: loss 0.012075
batch 1023: loss 0.008437
batch 1024: loss 0.054485
batch 1025: loss 0.041917
batch 1026: loss 0.017764
batch 1027: loss 0.070907
batch 1028: loss 0.010628
batch 1029: loss 0.111055
batch 1030: loss 0.003454
batch 1031: loss 0.

batch 1320: loss 0.004614
batch 1321: loss 0.008863
batch 1322: loss 0.061309
batch 1323: loss 0.036794
batch 1324: loss 0.058060
batch 1325: loss 0.083519
batch 1326: loss 0.004241
batch 1327: loss 0.015858
batch 1328: loss 0.050523
batch 1329: loss 0.098361
batch 1330: loss 0.088546
batch 1331: loss 0.077854
batch 1332: loss 0.137238
batch 1333: loss 0.001968
batch 1334: loss 0.071781
batch 1335: loss 0.001303
batch 1336: loss 0.009656
batch 1337: loss 0.037210
batch 1338: loss 0.053420
batch 1339: loss 0.038718
batch 1340: loss 0.037841
batch 1341: loss 0.011882
batch 1342: loss 0.087188
batch 1343: loss 0.173715
batch 1344: loss 0.002363
batch 1345: loss 0.075637
batch 1346: loss 0.004764
batch 1347: loss 0.346189
batch 1348: loss 0.029814
batch 1349: loss 0.062988
batch 1350: loss 0.011185
batch 1351: loss 0.079882
batch 1352: loss 0.016785
batch 1353: loss 0.022433
batch 1354: loss 0.006287
batch 1355: loss 0.017074
batch 1356: loss 0.092277
batch 1357: loss 0.035222
batch 1358: 

...

batch 4582: loss 0.032351
batch 4583: loss 0.000782
batch 4584: loss 0.088600
batch 4585: loss 0.003425
batch 4586: loss 0.059569
batch 4587: loss 0.000157
batch 4588: loss 0.111835
batch 4589: loss 0.001468
batch 4590: loss 0.031200
batch 4591: loss 0.007774
batch 4592: loss 0.005079
batch 4593: loss 0.062306
batch 4594: loss 0.013017
batch 4595: loss 0.007190
batch 4596: loss 0.014675
batch 4597: loss 0.000501
batch 4598: loss 0.052729
batch 4599: loss 0.000549
batch 4600: loss 0.014451
batch 4601: loss 0.000543
batch 4602: loss 0.026837
batch 4603: loss 0.000479
batch 4604: loss 0.020926
batch 4605: loss 0.007061
batch 4606: loss 0.101725
batch 4607: loss 0.007263
batch 4608: loss 0.000107
batch 4609: loss 0.000598
batch 4610: loss 0.001001
batch 4611: loss 0.000761
batch 4612: loss 0.022958
batch 4613: loss 0.000801
batch 4614: loss 0.043555
batch 4615: loss 0.000607
batch 4616: loss 0.008819
batch 4617: loss 0.000521
batch 4618: loss 0.001927
batch 4619: loss 0.017477
batch 4620: 

batch 4910: loss 0.001166
batch 4911: loss 0.000079
batch 4912: loss 0.001137
batch 4913: loss 0.000732
batch 4914: loss 0.012307
batch 4915: loss 0.011657
batch 4916: loss 0.018849
batch 4917: loss 0.000299
batch 4918: loss 0.000416
batch 4919: loss 0.000393
batch 4920: loss 0.002698
batch 4921: loss 0.000125
batch 4922: loss 0.001764
batch 4923: loss 0.008095
batch 4924: loss 0.003278
batch 4925: loss 0.003000
batch 4926: loss 0.002061
batch 4927: loss 0.000321
batch 4928: loss 0.011639
batch 4929: loss 0.002856
batch 4930: loss 0.013250
batch 4931: loss 0.001863
batch 4932: loss 0.000853
batch 4933: loss 0.000357
batch 4934: loss 0.005577
batch 4935: loss 0.012442
batch 4936: loss 0.004893
batch 4937: loss 0.002264
batch 4938: loss 0.037504
batch 4939: loss 0.001763
batch 4940: loss 0.031653
batch 4941: loss 0.003907
batch 4942: loss 0.000309
batch 4943: loss 0.020432
batch 4944: loss 0.000658
batch 4945: loss 0.000051
batch 4946: loss 0.000188
batch 4947: loss 0.005406
batch 4948: 

batch 5238: loss 0.002007
batch 5239: loss 0.011150
batch 5240: loss 0.000949
batch 5241: loss 0.000964
batch 5242: loss 0.009568
batch 5243: loss 0.000470
batch 5244: loss 0.002310
batch 5245: loss 0.006450
batch 5246: loss 0.008853
batch 5247: loss 0.000040
batch 5248: loss 0.000368
batch 5249: loss 0.001200
batch 5250: loss 0.000234
batch 5251: loss 0.000241
batch 5252: loss 0.000758
batch 5253: loss 0.001605
batch 5254: loss 0.000602
batch 5255: loss 0.000444
batch 5256: loss 0.021176
batch 5257: loss 0.000651
batch 5258: loss 0.002946
batch 5259: loss 0.013891
batch 5260: loss 0.000335
batch 5261: loss 0.001259
batch 5262: loss 0.000097
batch 5263: loss 0.001891
batch 5264: loss 0.000294
batch 5265: loss 0.010343
batch 5266: loss 0.000233
batch 5267: loss 0.043370
batch 5268: loss 0.001869
batch 5269: loss 0.001279
batch 5270: loss 0.012899
batch 5271: loss 0.003578
batch 5272: loss 0.036009
batch 5273: loss 0.000043
batch 5274: loss 0.011361
batch 5275: loss 0.005382
batch 5276: 

batch 5568: loss 0.000364
batch 5569: loss 0.041671
batch 5570: loss 0.011553
batch 5571: loss 0.002011
batch 5572: loss 0.014481
batch 5573: loss 0.002010
batch 5574: loss 0.021648
batch 5575: loss 0.000268
batch 5576: loss 0.009012
batch 5577: loss 0.001840
batch 5578: loss 0.002271
batch 5579: loss 0.000237
batch 5580: loss 0.000366
batch 5581: loss 0.001980
batch 5582: loss 0.126189
batch 5583: loss 0.032043
batch 5584: loss 0.000467
batch 5585: loss 0.002923
batch 5586: loss 0.002864
batch 5587: loss 0.001145
batch 5588: loss 0.000752
batch 5589: loss 0.000064
batch 5590: loss 0.002144
batch 5591: loss 0.001004
batch 5592: loss 0.005921
batch 5593: loss 0.002294
batch 5594: loss 0.000445
batch 5595: loss 0.007569
batch 5596: loss 0.000457
batch 5597: loss 0.000309
batch 5598: loss 0.001614
batch 5599: loss 0.017962
batch 5600: loss 0.017237
batch 5601: loss 0.034579
batch 5602: loss 0.000572
batch 5603: loss 0.001250
batch 5604: loss 0.001548
batch 5605: loss 0.000062
batch 5606: 

batch 5897: loss 0.031418
batch 5898: loss 0.002472
batch 5899: loss 0.000357
batch 5900: loss 0.000626
batch 5901: loss 0.003834
batch 5902: loss 0.000097
batch 5903: loss 0.000606
batch 5904: loss 0.000527
batch 5905: loss 0.012573
batch 5906: loss 0.001159
batch 5907: loss 0.009239
batch 5908: loss 0.032152
batch 5909: loss 0.001868
batch 5910: loss 0.001627
batch 5911: loss 0.000029
batch 5912: loss 0.003204
batch 5913: loss 0.000528
batch 5914: loss 0.002605
batch 5915: loss 0.001147
batch 5916: loss 0.006503
batch 5917: loss 0.001447
batch 5918: loss 0.003717
batch 5919: loss 0.000130
batch 5920: loss 0.000382
batch 5921: loss 0.029763
batch 5922: loss 0.001017
batch 5923: loss 0.000213
batch 5924: loss 0.009627
batch 5925: loss 0.001325
batch 5926: loss 0.001113
batch 5927: loss 0.004992
batch 5928: loss 0.001079
batch 5929: loss 0.174032
batch 5930: loss 0.064170
batch 5931: loss 0.000019
batch 5932: loss 0.027952
batch 5933: loss 0.009905
batch 5934: loss 0.000189
batch 5935: 

In [7]:
sparse_categorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
num_batches = int(data_loader.num_test_data // batch_size)
for batch_index in range(num_batches):
    start_index, end_index = batch_index * batch_size, (batch_index + 1) * batch_size
    y_pred = model.predict(data_loader.test_data[start_index: end_index])
    sparse_categorical_accuracy.update_state(y_true=data_loader.test_label[start_index: end_index], y_pred=y_pred)
print("test accuracy: %f" % sparse_categorical_accuracy.result())

test accuracy: 0.992300


<img src="https://tf.wiki/_images/cnn.png">

## 可以發現準確率相較於前節的多層感知器有非常顯著的提高。事實上，通過改變模型的網路結構（比如加入 Dropout 層防止過擬合），準確率還有進一步提升的空間。

# 使用 Keras 中預定義的典型卷積神經網路結構

## tf.keras.applications 中有一些預定義好的典型卷積神經網路結構，如 VGG16 、 VGG19 、 ResNet 、 MobileNet 等。我們可以直接呼叫這些典型的卷積神經網路結構（甚至載入預訓練的參數），而無需手動定義網路結構。